In [281]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)




In [282]:
# Display the data table for preview
mouse_metadata.head()



,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16


In [283]:
# Display the data table for preview
study_results.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [284]:
# Combine the data into a single dataset
# Merge two dataframes using an inner join
merged_mice_df = pd.merge(mouse_metadata, study_results, on="Mouse ID")
# Display the merged data table for preview
merged_mice_df.head()


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [285]:
#Check for rows w missing data
merged_mice_df.head()


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [286]:
#Verify data types
merged_mice_df.dtypes


Mouse ID               object
Drug Regimen           object
Sex                    object
Age_months              int64
Weight (g)              int64
Timepoint               int64
Tumor Volume (mm3)    float64
Metastatic Sites        int64
dtype: object

In [287]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
# Optional: Get all the data for the duplicate mouse ID. 
duped_mice = merged_mice_df[merged_mice_df.duplicated(['Mouse ID', 'Timepoint'])]
duped_mice

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
909,g989,Propriva,Female,21,26,0,45.000000,0
911,g989,Propriva,Female,21,26,5,47.570392,0
913,g989,Propriva,Female,21,26,10,49.880528,0
915,g989,Propriva,Female,21,26,15,53.442020,0
917,g989,Propriva,Female,21,26,20,54.657650,1


In [288]:
# Checking the number of mice in the clean DataFrame.
merged_mice_df['Mouse ID'].nunique()

249

In [289]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
# Get indexes where name column has value john
indexNames = merged_mice_df[merged_mice_df['Mouse ID'] == 'g989'].index
# Delete these row indexes from dataFrame
merged_mice_df.drop(indexNames , inplace=True)

In [290]:
# Checking the number of mice in the clean DataFrame.
merged_mice_df['Mouse ID'].nunique()

248

In [291]:
#Summary Stats
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method is the most straighforward, creating multiple series and putting them all together at the end.
#Reaname colume for ease of typing
merged_mice_df = merged_mice_df.rename(columns={'Tumor Volume (mm3)': 'Tumor Volume'})
#create the summary stats
var_merged_mice = merged_mice_df[['Drug Regimen', 'Tumor Volume']].groupby('Drug Regimen').var()
median_merged_mice = merged_mice_df[['Drug Regimen', 'Tumor Volume']].groupby('Drug Regimen').median()
sem_merged_mice = merged_mice_df[['Drug Regimen', 'Tumor Volume']].groupby('Drug Regimen').sem()
mean_merged_mice = merged_mice_df[['Drug Regimen', 'Tumor Volume']].groupby('Drug Regimen').mean()
stdev_merged_mice = merged_mice_df[['Drug Regimen', 'Tumor Volume']].groupby('Drug Regimen').std()
#Creat summary stat df to hold above stats
sumStas_merged_mice_df = pd.DataFrame(mean_merged_mice)
#reaname colume for ease of understanding
sumStas_merged_mice_df = sumStas_merged_mice_df.rename(columns={'Tumor Volume': 'Mean'})
#adding other items to the the DF
sumStas_merged_mice_df['Median'] = median_merged_mice
sumStas_merged_mice_df['Variance'] = var_merged_mice
sumStas_merged_mice_df['StDev'] = stdev_merged_mice
sumStas_merged_mice_df['SEM'] = sem_merged_mice
#display DF
sumStas_merged_mice_df




,Mean,Median,Variance,StDev,SEM
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [292]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function
other_sumStats = merged_mice_df[['Drug Regimen', 'Tumor Volume']].groupby('Drug Regimen').agg({'Tumor Volume' : ['mean', 'median', 'var', 'std', 'sem']})
other_sumStats

Tumor Volume                                          
                     mean     median        var       std       sem
Drug Regimen                                                       
Capomulin       40.675741  41.557809  24.947764  4.994774  0.329346
Ceftamin        52.591172  51.776157  39.290177  6.268188  0.469821
Infubinol       52.884795  51.820584  43.128684  6.567243  0.492236
Ketapril        55.235638  53.698743  68.553577  8.279709  0.603860
Naftisol        54.331565  52.509285  66.173479  8.134708  0.596466
Placebo         54.033581  52.288934  61.168083  7.821003  0.581331
Propriva        52.320930  50.446266  43.852013  6.622085  0.544332
Ramicane        40.216745  40.673236  23.486704  4.846308  0.320955
Stelasyn        54.233149  52.431737  59.450562  7.710419  0.573111
Zoniferol       53.236507  51.818479  48.533355  6.966589  0.516398

In [293]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 
#Find mice per timepoint
#mice_over_time = merged_mice_df[['Timepoint', 'Drug Regimen']].groupby('Timepoint')
mice_over_time = merged_mice_df
del mice_over_time['Sex']
del mice_over_time['Age_months']
del mice_over_time['Weight (g)']
del mice_over_time['Tumor Volume']
del mice_over_time['Metastatic Sites']
mice_over_time = mice_over_time.groupby(['Timepoint', 'Drug Regimen'])
mice_over_time_df = pd.DataFrame(mice_over_time.count())
mice_over_time_df


Mouse ID
Timepoint Drug Regimen          
0         Capomulin           25
          Ceftamin            25
          Infubinol           25
          Ketapril            25
          Naftisol            25
...                          ...
45        Placebo             11
          Propriva             7
          Ramicane            20
          Stelasyn            11
          Zoniferol           14

[100 rows x 1 columns]